In [13]:
ban='_1636619329.723694_1_0.6912_0.7004'
batch_size=4

In [2]:
import torch

device0 = torch.device('cuda:5' if torch.cuda.is_available() else "cpu")
device1 = torch.device('cuda:5' if torch.cuda.is_available() else "cpu")

In [3]:
# 读数据
import pandas as pd
df_train=pd.read_excel("../data/df_train.xlsx",index_col=0)
df_test=pd.read_excel("../data/df_test.xlsx",index_col=0)

In [4]:
import ast
label18=[]
for i in range(len(df_train)):
    new_list=[]
    old_list=ast.literal_eval(df_train.loc[i]['label'])
    df_train.loc[i]['label']=old_list
    for j in old_list:
        if j==0:
            new_list+=[0,0,0]
        if j==1:
            new_list+=[0,0,1]
        if j==2:
            new_list+=[0,1,1]
        if j==3:
            new_list+=[1,1,1]
    label18.append(new_list)

df_train['label18']=label18

In [5]:
label18=[]
for i in range(len(df_test)):
    new_list=[]
    old_list=ast.literal_eval(df_test.loc[i]['label'])
    df_test.loc[i]['label']=old_list
    for j in old_list:
        if j==0:
            new_list+=[0,0,0]
        if j==1:
            new_list+=[0,0,1]
        if j==2:
            new_list+=[0,1,1]
        if j==3:
            new_list+=[1,1,1]
    label18.append(new_list)

df_test['label18']=label18

In [6]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
df_train_text=[df_train['text'][i] for i in range(len(df_train))]
df_test_text=[df_test['text'][i] for i in range(len(df_test))]


In [7]:
text2id_train = tokenizer(
        df_train_text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
input_ids_train=text2id_train["input_ids"]
mask_train=text2id_train["attention_mask"]

text2id_test = tokenizer(
        df_test_text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
input_ids_test=text2id_test["input_ids"]
mask_test=text2id_test["attention_mask"]


In [8]:
df_train['input_ids']=input_ids_train.tolist()
df_train['mask']=mask_train.tolist()

df_test['input_ids']=input_ids_test.tolist()
df_test['mask']=mask_test.tolist()

In [9]:
from torch.utils.data import Dataset
class SentimentDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "text"]
        label = self.dataset.loc[idx, "label"]
#         print(label)
        input_ids = self.dataset.loc[idx, "input_ids"]
        mask = self.dataset.loc[idx, "mask"]
        label18=self.dataset.loc[idx, "label18"]
        sample = {"text": text, "label": label,"label18": label18,"input_ids":input_ids,"mask":mask}
        # print(sample)
        return sample

In [14]:
#按batch_size分
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import torch

train_loader = DataLoader(
    SentimentDataset(df_train), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0
)
print(train_loader)

test_loader = DataLoader(
    SentimentDataset(df_test), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)
print(test_loader)

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F


class fn_cls(nn.Module):
    def __init__(self,device):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-chinese")
        self.model.to(device)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, 18)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
        x = outputs[1]
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [19]:
sigmoid = nn.Sigmoid()

In [20]:
import torch

if ban==0:
    cls = fn_cls(device0)
    cls.train()
else:
    cls=torch.load("../data/cls"+str(ban)+".model",map_location=device0)

In [21]:
def _18to6_(output,round_=0):#18tensor 变 6list
    output0=[]
    for j in output:#18
        i=0
        s=''
        list6=[]
        while(i<len(j)):
            ok=j[i]*1.2+j[i+1]*1.1+j[i+2]*1
            if round_==1:
                list6.append(ok.round().int().tolist())
            else:
                list6.append(ok.tolist())
            i+=3
        output0.append(list6)
    return output0

def get_loss_test(output,A,round_=0):#8*18
    output_r=torch.Tensor(_18to6_(output,round_=1)).to(device0)
    output=_18to6_(output,round_=round_)
    # print(output_r,A)
    cor_add=(output_r == A).sum().item()
    sum0=0
    for i in range(len(output)):
        for j in range(6):
            sum0+=(output[i][j]-A[i][j])*(output[i][j]-A[i][j])
    return sum0,cor_add

def test(device_test):
    cls.to(device_test)
    cls.eval()

    correct = 0
    total = 0
    loss_test=0
    for batch_idx,batch in enumerate(test_loader):
        with torch.no_grad():
            # print(batch['label'],batch['label18'])
            label6=torch.stack(batch['label']).t().to(device_test).float()
            # label18=torch.stack(batch['label18']).t().to(device_test).float()

            input_ids=torch.stack(batch['input_ids']).t().to(device_test)
            mask=torch.stack(batch['mask']).t().to(device_test)

            output = cls(input_ids, attention_mask=mask)
            output=sigmoid(output)

            total += len(output)*6
            loss_add,cor_add = get_loss_test(output, label6 ,round_=0)
            loss_test+=loss_add
            correct+=cor_add

            tes_score=1/(1+(loss_test/total) ** 0.5)
            acc_score=100.*correct/total

        

        print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%，score：{}'.format(
                    batch_idx, len(test_loader),100.*batch_idx/len(test_loader), 
                    correct, total,acc_score,
                    tes_score
            ),end='\r')
    print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%，score：{}'.format(
                    batch_idx, len(test_loader),100.*batch_idx/len(test_loader), 
                    correct, total,acc_score,
                    tes_score
            ))
#     cls.to(device_train)
    return tes_score,acc_score

test(device1)

[1669/1670 (100%)]	正确分类的样本数：35199，样本总数：40080，准确率：87.82%，score：0.7145693898200989


(tensor(0.7146, device='cuda:5'), 87.82185628742515)

In [26]:
weight=[]
import numpy as np
sumn=np.zeros(18)
for i in range(len(df_train)):
    print(i,end='\r')
    sumn+=np.array(df_train.loc[i]['label18'])
#     print(sumn,end='\r')

print(sumn.sum())
print(sumn.tolist())
    

24585.0
[408.0, 722.0, 1132.0, 221.0, 646.0, 1868.0, 280.0, 752.0, 1566.0, 544.0, 1589.0, 2994.0, 378.0, 1202.0, 2130.0, 932.0, 2638.0, 4583.0]


In [34]:
weight=[i/sumn.sum() for i in sumn.tolist()]
print(weight)
weight=torch.Tensor(weight).to(device0)
print(weight)

[0.01659548505186089, 0.029367500508440107, 0.046044335977221885, 0.008989221069757983, 0.02627618466544641, 0.0759812894041082, 0.01138905836892414, 0.030587756762253407, 0.06369737644905431, 0.022127313402481186, 0.0646329062436445, 0.12178157413056742, 0.01537522879804759, 0.048891600569452916, 0.08663819402074435, 0.03790929428513321, 0.10730119991864959, 0.1864144803742119]
tensor([0.0166, 0.0294, 0.0460, 0.0090, 0.0263, 0.0760, 0.0114, 0.0306, 0.0637,
        0.0221, 0.0646, 0.1218, 0.0154, 0.0489, 0.0866, 0.0379, 0.1073, 0.1864],
       device='cuda:5')


In [35]:
from torch import optim
#BCEWithLogitsLoss就是把Sigmoid-BCELoss合成一步
criterion = nn.BCELoss(weight=weight)
criterion_hui=nn.MSELoss()
optimizer = optim.Adam(cls.parameters(), lr=1e-4)

In [36]:
import time  # 引入time模块
 

import torch
import matplotlib.pyplot as plt

def train(device_train,device_test,epoch_be=0):
    epoch_num=4
    accumulation_steps=32
    
    train_score=[]
    test_score=[]
    test_acc=[]

    for epoch_idx in range(epoch_be,epoch_be+epoch_num):
        cls.to(device_train)
        print("_________________________________________________________________")
        print("_________________________________________________________________")
        print("_________________ epoch:"+str(epoch_idx)+" start _________________")
        print("_________________________________________________________________")
        print("_________________________________________________________________")
        score_list=[]
        loss_a=0
        total=0
        correct=0
        start = time.time()
        for batch_idx,batch in enumerate(train_loader):
            label6=torch.stack(batch['label']).t().to(device_test).float()
            label18=torch.stack(batch['label18']).t().to(device_test).float()

            #计算output
            input_ids=torch.stack(batch['input_ids']).t().to(device_train)
            mask=torch.stack(batch['mask']).t().to(device_train)
            output = cls(input_ids, attention_mask=mask)
            output=sigmoid(output)
            output6=torch.Tensor(_18to6_(output,round_=0)).to(device_train)


            # 梯度积累
            loss0 = criterion(output, label18)
            lossh=criterion_hui(output6,label6)

            loss=loss0+lossh
            loss = loss/accumulation_steps
            loss.backward()

            #计算score
            with torch.no_grad():
                total += len(output)*6
                loss_add,cor_add = get_loss_test(output, label6,round_=0)
                loss_a+=loss_add
                correct+=cor_add

                tr_score=1/(1+(loss_a/total) ** 0.5)
                acc_score=100.*correct/total


            if((batch_idx+1) % accumulation_steps) == 0:
                # 每 accumulation_steps 次更新一下网络中的参数
                optimizer.step()
                optimizer.zero_grad()

            if ((batch_idx+1) % accumulation_steps) == 1:
                score_list.append(tr_score)
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tscore:{:.6f}\t准确率：{:.2f}%'.format(
                    epoch_idx, batch_idx, len(train_loader), 
                    100.*batch_idx/len(train_loader), tr_score,acc_score
                ),end='\r')

        #每个epoch结束：
        print('labels:', label6)
        print('pred:', output6)
#         plt.plot([i for i in range(len(score_list))], score_list)
#         plt.show()

        print('Train Epoch: {}\tscore:{:.6f}\t准确率：{:.2f}%'.format(epoch_idx,tr_score,acc_score))
        t_score,t_acc=test(device_test)

        train_score.append(tr_score)
        test_score.append(t_score)
        test_acc.append(t_acc)

        #保存模型
        end = time.time()
        print('epoch:',str(epoch_idx)+'_执行时间: ',end - start)
        torch.save(cls,"../data/cls_"+str(end)+"_"+str(epoch_idx)+"_"+str(round(tr_score.tolist(),4))+"_"+str(round(t_score.tolist(),4))+".model")

        print("_________________________________________________________________")
        print("_________________________________________________________________")
        print("_________________ epoch:"+str(epoch_idx)+" end _________________")
        print("_________________________________________________________________")
        print("_________________________________________________________________")

#     plt.plot([i for i in range(len(train_score))], train_score)
#     plt.show()             

#     plt.plot([i for i in range(len(test_score))], test_score)
#     plt.show()

#     plt.plot([i for i in range(len(test_acc))], test_acc)
#     plt.show()
    
train(device0,device1,epoch_be=0)


_________________________________________________________________
_________________________________________________________________
_________________ epoch:0 start _________________
_________________________________________________________________
_________________________________________________________________
labels: tensor([[0., 0., 0., 0., 0., 0.],.642422	准确率：84.55%
        [0., 0., 0., 0., 0., 0.]], device='cuda:5')
pred: tensor([[0.0374, 0.0875, 0.0270, 0.1245, 0.0690, 0.2045],
        [0.0415, 0.0382, 0.0439, 0.1128, 0.0760, 0.0953]], device='cuda:5')
Train Epoch: 0	score:0.642436	准确率：84.55%
labels: tensor([[0., 0., 0., 0., 0., 0.],.691196	准确率：90.34%.7422773241996765
        [0., 0., 0., 2., 0., 0.]], device='cuda:5')
pred: tensor([[0.0453, 0.0224, 0.0362, 0.0236, 0.0740, 0.0415],
        [0.0378, 0.0310, 0.0542, 0.9821, 0.1138, 0.2330]], device='cuda:5')
Train Epoch: 1	score:0.691189	准确率：90.34%
[1669/1670 (100%)]	正确分类的样本数：35199，样本总数：40080，准确率：87.82%，score：0.7003803849220276
ep

In [31]:
end = time.time()
torch.save(cls,"../data/cls_"+str(end)+"_"+str(2)+"_"+str(0.6886)+"_"+str(0.6942)+".model")


In [35]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

def get_output(device_pre,model,text,f=0):
    model.to(device_pre)
    model.eval()
    text2id = tokenizer(
        text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )
#     print(text,text2id)
    input_ids=text2id["input_ids"].to(device_pre)
    mask=text2id["attention_mask"].to(device_pre)
#         print(text2id)
    output = model(input_ids, attention_mask=mask)
    print(output)
    output=sigmoid(output)
    print(output)
    
#     output=sigmoid(output)*4
    if f==1:
        return _18to6_(output,round_=1)
    else:
        return _18to6_(output,round_=0)

In [36]:
text = ['*与n3：啊？']
# 爱、乐、惊、怒、恐、哀
print(get_output(device1,cls,text,0))

tensor([[-6.9952, -6.0604, -5.4647, -7.4934, -6.1120, -5.5436, -2.6384, -0.2311,
          1.5884, -6.6223, -5.7481, -4.5401, -5.4886, -4.1617, -2.5640, -6.0817,
         -5.3633, -4.8412]], device='cuda:5', grad_fn=<AddmmBackward0>)
tensor([[9.1540e-04, 2.3281e-03, 4.2157e-03, 5.5645e-04, 2.2112e-03, 3.8972e-03,
         6.6708e-02, 4.4248e-01, 8.3039e-01, 1.3286e-03, 3.1787e-03, 1.0560e-02,
         4.1168e-03, 1.5342e-02, 7.1493e-02, 2.2791e-03, 4.6637e-03, 7.8357e-03]],
       device='cuda:5', grad_fn=<SigmoidBackward0>)
[[0.007875097915530205, 0.006997216492891312, 1.3971621990203857, 0.01565060205757618, 0.09330970048904419, 0.015700681135058403]]
